## MLE estimation of a simple macroeconomic model

You can observe time series data in an economy for the following variables $(c_t, k_t, w_t, r_t)$. The first line of this file contains variable labels. These data have 100 periods, which are quarterly (25 years). Suppose you think the data are generated by a pricess similar to the Brock and Mirman (1972). A simplified set of characterizing equations of the Brock and Mirman model are the following:
\begin{align}
c_t^{-1} - \beta E[r_{t+1}(c_{t+1})^{-1}] & = 0 && \text{(1)} \\
c_t - k_{t+1} - w_t - r_t k_t & = 0 && \text{(2)} \\
w_t - (1-\alpha)e^{z_t}k_t^\alpha & = 0 && \text{(3)} \\
r_t - \alpha e^{z_t} k_t^{\alpha - 1} & = 0 && \text{(4)} \\
z_t & = \rho z_{t-1} + (1-\rho)\mu + \epsilon_t && \text{(5)} \\
\epsilon_t & \sim N(0,\sigma^2)
\end{align}
The variable $c_t$ is aggregate consumption in period $t$. $k_{t+1}$ is total household savings and investment in period $t$ for which they receive a return in the next period (this model assumes full depreciation of capital). The wage per unit of labor in period $t$ is $w_t$ adn the interest rate or rate of return on investment is $r_t$. Total factor productivity is $z_t$, which follows an AR(1) process. The rest of the symbols in the equations are parameters that must be estimated:
$$ (\alpha, \beta, \rho, \mu, \sigma) $$
The constraints on these parameters are:
$$ \alpha, \beta \in (0,1), \quad \mu,\sigma > 0, \quad \rho \in (-1,1) $$
Assume that the first observation in the data file is $t=1$. Let $k_1$ be the first observation in the data file for $k_t$. Assume that $z_0 = \mu$, so that $z_1 = \mu$. Assume that the discount factor is known to be $\beta = 0.99$.

In [8]:
# Load Modules
import numpy as np
import scipy.stats as sts
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
import scipy.special as spc
import scipy.integrate as integrate

In [19]:
# Import Data
macroseries = pd.read_csv('MacroSeries.csv')

In [22]:
macroseries["c_t"]

0     10700000.0
1     11700000.0
2      9290000.0
3      7910000.0
4      8140000.0
5      8270000.0
6      8630000.0
7      9010000.0
8      8790000.0
9      9720000.0
10    10100000.0
11     9720000.0
12    11000000.0
13    13400000.0
14    11700000.0
15    10700000.0
16    11000000.0
17    10100000.0
18     8230000.0
19     9120000.0
20     8930000.0
21     8170000.0
22     7310000.0
23     7770000.0
24     8270000.0
25     7790000.0
26     7200000.0
27     8210000.0
28     8900000.0
29     8780000.0
         ...    
70     9970000.0
71    10100000.0
72    11700000.0
73    12300000.0
74    11800000.0
75    10200000.0
76     9390000.0
77     8410000.0
78     8930000.0
79     9430000.0
80     9420000.0
81    10300000.0
82     9590000.0
83    10500000.0
84    10800000.0
85    11900000.0
86    12700000.0
87    13900000.0
88    12000000.0
89    11600000.0
90    11000000.0
91    10800000.0
92    12100000.0
93    12600000.0
94    12000000.0
95    12200000.0
96    15300000.0
97    14700000

(a) Use the data $(w_t,k_t)$ and equations (3) and (5) to estimate the four parameters $(\alpha, \rho, \mu, \sigma)$ by maximum likelihood. Given a guess for the parameters $(\alpha, \rho, \mu, \sigma)$, you can use the two variables from the data $(w_t,k_t)$ and (3) to back out a series for $z_t$. You can then use equation (5) to compute the probability of each 
$$ z_t \sim N(\rho z_{t-1} + (1-\rho) \mu, \sigma^2) $$
The maximum likelihood estimate $(\hat{\alpha},\hat{\rho},\hat{\mu},\hat{\sigma})$ maximzies the likelihood function of that normal distribution of $z_t$'s. Report your estimates and the inverse hessian variance-covariance matrix of your estimates.

(b) Use the data $(r_t,k_t)$ and equations (4) and (5) to estimate the four parameters $(\alpha, \rho, \mu, \sigma)$ by maximum likelihood. Given a guess for the parameters $(\alpha, \rho, \mu, \sigma)$, you can use the two variables from the data $(r_t,k_t)$ and (4) to back out a series for $z_t$. You can then use equation (5) to compute the probability of each 
$$ z_t \sim N(\rho z_{t-1} + (1-\rho) \mu, \sigma^2) $$
The maximum likelihood estimate $(\hat{\alpha},\hat{\rho},\hat{\mu},\hat{\sigma})$ maximzies the likelihood function of that normal distribution of $z_t$'s. Report your estimates and the inverse hessian variance-covariance matrix of your estimates.

(c) According to your estimates from part (a), if investment/savings today is $k_t = 7,500,000$ and the productivity shock today is $z_{t-1} = 10$, what is the probability that the interest rate this period will be greater than $r_t =1$. That is, solve for $Pr(r_t > 1 | \hat{\theta},k_t,c_t, z_{t-1})$. [HINT: Use equation (4) to solve for the $z_t = z^*$ such that $r_t = 1$. Then use (5) to solve for the probability that $z_t > z^*$]